In [15]:
import os
import random
import pickle
import torch
import torchvision
import pandas as pd
import numpy as np

In [18]:
import torchvision.transforms as transforms

In [6]:
torch.cuda.is_available()

True

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)
torch.backends.cudnn.deterministic = True

In [11]:
class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.view(x.shape[0], -1)

In [12]:
with open('../pickles_for_git/model_6051.p', 'rb') as f:
    model = pickle.load(f)

In [26]:
with open('../pickles_for_git/main_dict.pickle', 'rb') as f:
    main_dict = pickle.load(f)

In [16]:
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

In [20]:
train_dir = '../data/spectrograms/train/train/'
val_dir = '../data/spectrograms/train/val/'
test_dir = '../data/spectrograms/test/'

In [88]:
one_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize([0.0069], [0.0033])
])

train_dataset = ImageFolderWithPaths(train_dir, one_transform)
val_dataset = ImageFolderWithPaths(val_dir, one_transform)

batch_size = 4
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=8
)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=8
)

class_names = train_dataset.classes

In [22]:
test_dataset = ImageFolderWithPaths(test_dir, one_transform)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=8
)

In [23]:
def predict(model, test_loader):
    with torch.no_grad():
        logits = []
    
        for inputs, x, path in test_loader:
            inputs = inputs.to(device)
            model.eval()
            outputs = model(inputs).cpu()
            logits.append(outputs)
            
    probs = torch.nn.functional.softmax(torch.cat(logits), dim=-1).numpy()
    return probs

In [24]:
probs = predict(model, test_dataloader)
preds = np.argmax(probs, axis=1)
preds = [class_names[x]for x in preds]

In [27]:
true_class = list()
for x in test_dataset:
    path = x[2]
    name = path.split('/')[5].strip('.png')
    genre = main_dict[int(name)]['genre']
    true_class.append(genre)

In [28]:
np.array(np.array(true_class) == np.array(preds)).sum() / len(preds)

0.7010309278350515

In [34]:
from sklearn.metrics import f1_score
f1_score(true_class, preds, average='weighted')

0.7012677559517543

In [37]:
with torch.no_grad():
    logits = []

    for inputs, x, path in test_dataloader:
        inputs = inputs.to(device)
        model.eval()
        outputs = model(inputs).cpu()
        logits.append(outputs)
        break

In [38]:
outputs.shape

torch.Size([4, 8])

In [42]:
outputs[3]

tensor([3.9162e-06, 2.7901e-03, 4.8492e-11, 2.3906e-06, 7.8391e-09, 1.2519e-05,
        5.9468e-05, 9.9998e-01])

In [50]:
flat_model = model[:-2]

In [85]:
with torch.no_grad():
    logits = []
    path_list = list()
    for inputs, x, path in test_dataloader:
        inputs = inputs.to(device)
        flat_model.eval()
        outputs = flat_model(inputs).cpu()
        logits.append(outputs)
        path_list.append(path)

In [89]:
with torch.no_grad():
#     logits = []
#     path_list = list()
    for inputs, x, path in val_dataloader:
        inputs = inputs.to(device)
        flat_model.eval()
        outputs = flat_model(inputs).cpu()
        logits.append(outputs)
        path_list.append(path)

In [91]:
with torch.no_grad():
#     logits = []
#     path_list = list()
    for inputs, x, path in train_dataloader:
        inputs = inputs.to(device)
        flat_model.eval()
        outputs = flat_model(inputs).cpu()
        logits.append(outputs)
        path_list.append(path)

In [92]:
len(path_list)

2000

In [82]:
path_list[0][0]

'../data/spectrograms/test/unknown/000190.png'

In [83]:
logits[0][0]

tensor([0.0000, 1.9347, 8.2890,  ..., 2.3243, 2.3456, 2.4686])

In [99]:
emb_list = list()
for i in logits:
    for tens in i:
        emb_list.append(tens.numpy())

In [102]:
embedings = pd.DataFrame(emb_list)

In [103]:
track_list = list()
for i in path_list:
    for path in i:
        name = path.split('/')[-1]
        ids = name.split('.')[0]
        track_list.append(ids)

In [106]:
embedings['id'] = track_list

In [107]:
embedings

,0,1,2,3,4,5,6,7,8,9,...,2039,2040,2041,2042,2043,2044,2045,2046,2047,id
0,0.000000,1.934654,8.288989,0.911272,0.000000,0.836374,2.723937,3.236278,0.196896,0.136825,...,8.378351,6.188560,2.416295,3.059616,8.658876,0.0,2.324260,2.345611,2.468582,000190
1,0.570574,0.906143,7.809146,0.685275,0.025747,0.831038,0.122490,0.159591,0.213468,0.715388,...,5.331222,6.732090,2.042198,4.235939,6.969593,0.0,3.396771,1.152505,4.091938,000194
2,0.610264,3.641630,2.293852,3.537972,2.972309,1.229408,0.385535,0.513138,1.706599,4.169592,...,2.355113,5.586826,0.000000,4.113567,3.054376,0.0,4.640083,2.106162,3.239305,000667
3,4.522550,4.245885,4.302017,2.959141,4.187956,0.454045,3.671622,4.877146,1.351015,5.414358,...,5.870000,1.112780,0.000000,6.024055,0.767975,0.0,7.130897,1.967405,2.922061,001040
4,0.972875,3.906113,5.861447,0.201382,0.168771,0.000000,5.051270,3.065430,1.117101,4.355967,...,2.962019,1.074586,3.008203,0.712963,4.901093,0.0,1.748662,4.261450,1.371169,001686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7989,0.937852,3.475553,2.832040,3.414687,4.226087,1.560897,0.403958,0.000000,1.562922,2.878870,...,6.187293,6.350399,0.000000,5.330186,0.872463,0.0,7.229657,2.248746,2.465659,149416
7990,0.442045,2.818931,2.934993,2.895631,4.263456,2.120693,1.922541,2.216130,0.581613,3.662995,...,7.112364,4.539965,0.456817,4.598337,0.000000,0.0,6.543571,1.480223,2.564905,149417
7991,3.401756,9.156358,5.577356,3.741688,4.383796,0.521133,0.299287,0.623174,3.219270,4.339592,...,4.736887,4.945627,0.000000,7.103370,2.790202,0.0,8.010686,1.356030,2.773219,149452
7992,2.623586,0.000000,6.798588,3.333807,3.605097,0.500432,0.080584,0.000000,3.436022,2.563543,...,5.750184,2.148381,2.091455,6.100538,0.633380,0.0,6.476285,0.000000,2.936096,149488


In [109]:
embedings = embedings[['id'] + list(np.arange(2048))]

In [110]:
embedings.to_csv('../data/csv/cnn_embedings.csv', index=False)